In [6]:
import pandas as pd
import numpy as np

import sqlite3
# Set random seed for reproducibility
np.random.seed(42)


# ETL

In [23]:
# Create SQLite database
conn = sqlite3.connect('guest_rooms.db')
cursor = conn.cursor()

In [25]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS rooms (
    room_id TEXT PRIMARY KEY,
    level TEXT NOT NULL,
    availability TEXT NOT NULL,
    room_type TEXT NOT NULL
)
''')

#### DELETE

In [24]:
cursor.execute(
    '''
    DROP TABLE rooms;

    '''
)

## Rooms

In [28]:

# Create room distribution by level
levels = {
    'level_1': 4,
    'level_2': 8,
    'level_3': 8
}

# Create lists of possible values
#heights = ['Low', 'Medium', 'High']
availabilities = ['available', 'occupied']
room_types = ['standard', 'deluxe', 'suite']

# Generate the data
data = []
room_counter = 1

for level, count in levels.items():
    for i in range(count):
        room_data = {
            'Room_ID': f"R{room_counter:03d}",
            'Level': level,
            'Availability': np.random.choice(availabilities, p=[0.6, 0.4] ),
            'Type': np.random.choice(room_types, p=[0.6, 0.3, 0.1])
        }
        data.append(room_data)
        room_counter += 1

# Create DataFrame
rooms_df = pd.DataFrame(data)
rooms_df.columns = [c.lower() for c in rooms_df.columns]
#
# # Set specific height distributions by level (optional)
# rooms_df.loc[rooms_df['Level'] == 'Level_1', 'Height'] = 'Low'
# rooms_df.loc[rooms_df['Level'] == 'Level_2', 'Height'] = 'Medium'
# rooms_df.loc[rooms_df['Level'] == 'Level_3', 'Height'] = 'High'

# Display the DataFrame
print(rooms_df.head(10))
print("\nRoom Count by Level:")
print(rooms_df['level'].value_counts())

  room_id    level availability      type
0    R001  level_1    available    deluxe
1    R002  level_1     occupied  standard
2    R003  level_1    available    deluxe
3    R004  level_1    available  standard
4    R005  level_2    available  standard
5    R006  level_2     occupied  standard
6    R007  level_2    available  standard
7    R008  level_2     occupied  standard
8    R009  level_2     occupied  standard
9    R010  level_2     occupied  standard

Room Count by Level:
level
level_2    8
level_3    8
level_1    4
Name: count, dtype: int64


In [29]:
rooms_df

,room_id,level,availability,type
0,R001,level_1,available,deluxe
1,R002,level_1,occupied,standard
2,R003,level_1,available,deluxe
3,R004,level_1,available,standard
4,R005,level_2,available,standard
5,R006,level_2,occupied,standard
6,R007,level_2,available,standard
7,R008,level_2,occupied,standard
8,R009,level_2,occupied,standard
9,R010,level_2,occupied,standard


### INSERT DATA

In [30]:
# Insert data using DataFrame iteration
for index, row in rooms_df.iterrows():
    cursor.execute('''
    INSERT INTO rooms (room_id, level, availability, room_type)
    VALUES (?, ?, ?, ?)
    ''', (row['room_id'], row['level'], row['availability'], row['type']))

# Commit changes
conn.commit()

# Verify data insertion
print("Total rooms inserted:", cursor.execute("SELECT COUNT(*) FROM rooms").fetchone()[0])
print("\nSample rooms:")
for row in cursor.execute("SELECT * FROM rooms LIMIT 5"):
    print(row)

# Close connection
conn.close()

Total rooms inserted: 20

Sample rooms:
('R001', 'level_1', 'available', 'deluxe')
('R002', 'level_1', 'occupied', 'standard')
('R003', 'level_1', 'available', 'deluxe')
('R004', 'level_1', 'available', 'standard')
('R005', 'level_2', 'available', 'standard')
